In [150]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Import library yang dibutuhkan

In [151]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Membaca dataset yang dengan ekstensi .csv dan menyimpannya ke dalam dataframe df

In [152]:
df = pd.read_csv("/content/drive/My Drive/Coolyeah/Semester 5/Dicoding ML/Machine Learning Terapan/Datasets/spotify_millsongdata.csv")
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


mengambil 5.000 baris pertama dari dataset

In [153]:
df = df[0:5000]

membersihkan dan mengubah struktur DataFrame lagu dengan menghapus kolom yang tidak diperlukan, mengganti nama kolom, menghapus duplikat berdasarkan judul lagu, dan mereset indeks untuk mempersiapkan data

In [154]:
# Menghapus Kolom "link"
df.drop(["link"], axis = 1, inplace = True)

# Mengganti Nama Kolom "text" Menjadi "lyrics"
df.rename(columns = {"text":"lyrics"}, inplace = True)

# Menghapus Duplikat Berdasarkan Kolom "song"
df.drop_duplicates(subset="song", inplace=True)

# Mereset Indeks DataFrame
df.reset_index(drop = True, inplace = True)

menampilkan df

In [155]:
df

,artist,song,lyrics
0,ABBA,Ahe's My Kind Of Girl,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante","Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,Making somebody happy is a question of give an...
...,...,...,...
4724,Elvis Costello,Love For Tender,You won't take my love for tender \r\nYou can...
4725,Elvis Costello,Love Went Mad,I've looked at it every way I can \r\nFrom un...
4726,Elvis Costello,Lover's Walk,I won't walk with my head bowed \r\n(Be on) B...
4727,Elvis Costello,Luxembourg,Dressed up like a dog's dinner \r\nButter wou...


menampilkan info df

In [156]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4729 entries, 0 to 4728
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   artist  4729 non-null   object
 1   song    4729 non-null   object
 2   lyrics  4729 non-null   object
dtypes: object(3)
memory usage: 111.0+ KB


membuat kolom baru "combined_features" yang menggabungkan informasi dari kolom "artist", "song", dan "lyrics" dalam dataframe

In [157]:
df["combined_features"] = df["artist"]+ " "+df["song"]+" "+df["lyrics"]

menampilkan df

In [158]:
df

,artist,song,lyrics,combined_features
0,ABBA,Ahe's My Kind Of Girl,"Look at her face, it's a wonderful face \r\nA...","ABBA Ahe's My Kind Of Girl Look at her face, i..."
1,ABBA,"Andante, Andante","Take it easy with me, please \r\nTouch me gen...","ABBA Andante, Andante Take it easy with me, pl..."
2,ABBA,As Good As New,I'll never know why I had to go \r\nWhy I had...,ABBA As Good As New I'll never know why I had ...
3,ABBA,Bang,Making somebody happy is a question of give an...,ABBA Bang Making somebody happy is a question ...
4,ABBA,Bang-A-Boomerang,Making somebody happy is a question of give an...,ABBA Bang-A-Boomerang Making somebody happy is...
...,...,...,...,...
4724,Elvis Costello,Love For Tender,You won't take my love for tender \r\nYou can...,Elvis Costello Love For Tender You won't take ...
4725,Elvis Costello,Love Went Mad,I've looked at it every way I can \r\nFrom un...,Elvis Costello Love Went Mad I've looked at it...
4726,Elvis Costello,Lover's Walk,I won't walk with my head bowed \r\n(Be on) B...,Elvis Costello Lover's Walk I won't walk with ...
4727,Elvis Costello,Luxembourg,Dressed up like a dog's dinner \r\nButter wou...,Elvis Costello Luxembourg Dressed up like a do...


menampilkan nilai dari kolom "combined_features" pada baris pertama dari dataframe

In [159]:
df.combined_features[0]

"ABBA Ahe's My Kind Of Girl Look at her face, it's a wonderful face  \r\nAnd it means something special to me  \r\nLook at the way that she smiles when she sees me  \r\nHow lucky can one fellow be?  \r\n  \r\nShe's just my kind of girl, she makes me feel fine  \r\nWho could ever believe that she could be mine?  \r\nShe's just my kind of girl, without her I'm blue  \r\nAnd if she ever leaves me what could I do, what could I do?  \r\n  \r\nAnd when we go for a walk in the park  \r\nAnd she holds me and squeezes my hand  \r\nWe'll go on walking for hours and talking  \r\nAbout all the things that we plan  \r\n  \r\nShe's just my kind of girl, she makes me feel fine  \r\nWho could ever believe that she could be mine?  \r\nShe's just my kind of girl, without her I'm blue  \r\nAnd if she ever leaves me what could I do, what could I do?\r\n\r\n"

 mencetak nilai dari kolom "combined_features" untuk lima baris pertama dari dataframe

In [160]:
for i in range(0,5):
    print("*********************************************\n")
    print(df.combined_features[i])

*********************************************

ABBA Ahe's My Kind Of Girl Look at her face, it's a wonderful face  
And it means something special to me  
Look at the way that she smiles when she sees me  
How lucky can one fellow be?  
  
She's just my kind of girl, she makes me feel fine  
Who could ever believe that she could be mine?  
She's just my kind of girl, without her I'm blue  
And if she ever leaves me what could I do, what could I do?  
  
And when we go for a walk in the park  
And she holds me and squeezes my hand  
We'll go on walking for hours and talking  
About all the things that we plan  
  
She's just my kind of girl, she makes me feel fine  
Who could ever believe that she could be mine?  
She's just my kind of girl, without her I'm blue  
And if she ever leaves me what could I do, what could I do?


*********************************************

ABBA Andante, Andante Take it easy with me, please  
Touch me gently like a summer evening breeze  
Take your time, m

Fungsi cleaning melakukan pembersihan teks dengan menghapus karakter non-alphabetic, mengubah teks menjadi huruf kecil, memisahkan kata-kata, dan menghapus kata-kata penghubung (stopwords) dalam bahasa Inggris.

In [161]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
def cleaning(text):
    text = re.sub("[^a-zA-Z]", " ", text)
    text = text.lower()
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


mengaplikasikan fungsi cleaning pada setiap nilai dalam kolom "combined_features" dataframe

In [162]:
df["combined_features"] = df["combined_features"].apply(func = cleaning)

mencetak nilai dari kolom "combined_features" untuk lima baris pertama DataFrame df setelah penerapan fungsi pembersihan teks

In [163]:
for i in range(0,5):
    print("\n*********************************************\n")
    print(df.combined_features[i])


*********************************************

abba ahe kind girl look face wonderful face means something special look way smiles sees lucky one fellow kind girl makes feel fine could ever believe could mine kind girl without blue ever leaves could could go walk park holds squeezes hand go walking hours talking things plan kind girl makes feel fine could ever believe could mine kind girl without blue ever leaves could could

*********************************************

abba andante andante take easy please touch gently like summer evening breeze take time make slow andante andante let feeling grow make fingers soft light let body velvet night touch soul know andante andante go slowly music music song song music song play time time make strong play cause making strong make sing make sound make sing make andante andante tread lightly ground andante andante oh please let shimmer eyes like feeling thousand butterflies please talk go play andante andante let float away music music song 

menampilkan "combined_features" pada baris pertama dari dataframe df setelah penerapan fungsi pembersihan teks

In [164]:
df.combined_features[0]

'abba ahe kind girl look face wonderful face means something special look way smiles sees lucky one fellow kind girl makes feel fine could ever believe could mine kind girl without blue ever leaves could could go walk park holds squeezes hand go walking hours talking things plan kind girl makes feel fine could ever believe could mine kind girl without blue ever leaves could could'

membuat objek TfidfVectorizer dengan batas maksimal fitur sebanyak 10,000, yang akan digunakan untuk mengonversi teks menjadi representasi vektor TF-IDF.

In [165]:
tfv = TfidfVectorizer(max_features = 10000)

mengonversi teks pada kolom "combined_features" DataFrame df menjadi matriks TF-IDF

In [166]:
tfv_matrix = tfv.fit_transform(df["combined_features"])

menampilkan shape matrix

In [167]:
tfv_matrix.shape

(4729, 10000)

menghitung matriks cosinus similarity berdasarkan matriks TF-IDF

In [168]:
cosine_sim = cosine_similarity(tfv_matrix)

In [169]:
cosine_sim

array([[1.        , 0.00436727, 0.04955508, ..., 0.0274748 , 0.02643377,
        0.12211386],
       [0.00436727, 1.        , 0.01649985, ..., 0.00201614, 0.00859367,
        0.00354659],
       [0.04955508, 0.01649985, 1.        , ..., 0.01672257, 0.01622956,
        0.07635679],
       ...,
       [0.0274748 , 0.00201614, 0.01672257, ..., 1.        , 0.01650344,
        0.0169903 ],
       [0.02643377, 0.00859367, 0.01622956, ..., 0.01650344, 1.        ,
        0.08413642],
       [0.12211386, 0.00354659, 0.07635679, ..., 0.0169903 , 0.08413642,
        1.        ]])

mencari indeks (index) dari lagu dengan judul "Hope" dalam DataFrame df dan menyimpannya dalam variabel song_index

In [170]:
song_user_likes = "Hope"
song_index = df[df.song == song_user_likes].index[0]

In [171]:
song_index

659

membuat daftar yang berisi indeks dan nilai kemiripan kosinus antara lagu "Hope" dengan setiap lagu lain dalam dataset

In [172]:
similar_songs = list(enumerate(cosine_sim[song_index]))

enampilkan lima elemen pertama dari daftar yang berisi pasangan indeks dan nilai cosinus similarity antara lagu "Hope" dengan lima lagu lainnya dalam dataset.

In [173]:
similar_songs[0:5]

[(0, 0.03951386606631649),
 (1, 0.020285614966625676),
 (2, 0.049503715758874973),
 (3, 0.0067000373901200255),
 (4, 0.0072045284322823)]

Kode tersebut mengurutkan daftar similar_songs berdasarkan nilai kemiripan kosinus secara menurun, dengan hasil yang diurutkan disimpan dalam variabel sorted_similar_songs.

In [174]:
sorted_similar_songs = sorted(similar_songs, key=lambda x:x[1], reverse=True)

menampilkan lima elemen pertama dari daftar yang berisi pasangan indeks dan nilai kemiripan kosinus antara lagu "Hope" dengan lagu-lagu lain dalam dataset, yang diurutkan berdasarkan tingkat kemiripan secara menurun.

In [175]:
sorted_similar_songs[0:5]

[(659, 1.0000000000000002),
 (3105, 0.3521985970415129),
 (2506, 0.3378953353536083),
 (4702, 0.31930153403873973),
 (1520, 0.31156719143137)]

mencetak judul sepuluh lagu yang paling mirip dengan lagu "Hope" berdasarkan nilai kemiripan kosinus yang telah diurutkan sebelumnya.

In [176]:
for song in sorted_similar_songs[1:11]:
        similar_songs = df[df.index == song[0]]["song"].values[0]
        print(similar_songs)

It's Alright Now
Anyway You Want
I Hope You're Happy Now
I'm Fine
Light Of Hope
Like Tonight
Perfume
(I Love You) For Sentimental Reasons
Above The Clouds
You're So Beautiful


Fungsi recommended_songs tersebut menerima judul lagu yang disukai oleh pengguna, menghitung kemiripan kosinus dengan lagu-lagu lain dalam dataset, dan mencetak judul sepuluh lagu yang paling mirip dengan lagu yang disukai tersebut.

In [177]:
def recommended_songs(song_user_likes):
    song_index = df[df.song == song_user_likes].index[0]
    similar_songs = list(enumerate(cosine_sim[song_index]))
    sorted_similar_songs = sorted(similar_songs, key=lambda x:x[1], reverse=True)

    for song in sorted_similar_songs[1:11]:
        similar_songs = df[df.index == song[0]]["song"].values[0]
        print(similar_songs)

menggunakan fungsi recommended_songs dengan parameter "Love", yang akan menghasilkan rekomendasi sepuluh lagu yang paling mirip dengan lagu "Love"

In [178]:
recommended_songs("Love")

I Do Love You
Mystery Of Love
I Love Her I Love Her
You Wrote The Book On Love
Love To Love You Baby
Who Do You Love
I Feel Love
This Is My Love
Some Love
Love Is Made Of This
